# Scrap The Data From Amazon

We need to intall Selenium.
Cammand for install selenium and also install a beutifulSoup



# Requrements:
# 1
first off all you download the web driver. For example if you are working on chrome so download the chrome web driver from       here and copy its path in envirnment.>    https://chromedriver.chromium.org/downloads
 also check the version of your chrome browser.
 
# 2
than download the selenium 
link>>just type pip install selenium
reference link>>>
https://stackoverflow.com/questions/46137219/how-to-install-selenium-in-a-conda-environment
 
 

In [3]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [1]:
import csv
from bs4 import BeautifulSoup

In [2]:
from selenium import webdriver

In [3]:
from msedge.selenium_tools import Edge, EdgeOptions

# Stratup The Web Driver

In [50]:
driver=webdriver.Chrome()

In [51]:
url='https:www.amazon.com'



In [52]:
driver.get(url)

In [7]:
def get_url(search_term):
    template='https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term=search_term.replace(' ','+')
    return template.format(search_term)

In [9]:
url=get_url('smart phone')
print(url)

https://www.amazon.com/s?k=smart+phone&ref=nb_sb_noss_2


In [10]:
driver.get(url)

# Extract the collection

In [12]:
soup=BeautifulSoup(driver.page_source, 'html.parser')

In [15]:
results=soup.find_all('div',{'data-component-type': 's-search-result'})

In [16]:
len(results)

22

# Create prototype of the Data

In [17]:
item=results[0]

In [18]:
atag=item.h2.a

In [20]:
description=atag.text.strip()

In [22]:
url='http://wwww.amazon.com'+atag.get('href')

In [23]:
price_class=item.find('span', 'a-price')

In [24]:
price=price_class.find('span','a-offscreen')

In [27]:
price=price.text

In [30]:
rating=item.i.text

In [33]:
review_count=item.find('span',{'class': 'a-size-base', 'dir':'auto'}).text


# Generelize the prototype

In [34]:
def extract_record(item):
    atag=item.h2.a
    description=atag.text.strip()
    url='http://wwww.amazon.com'+atag.get('href')
    price_class=item.find('span', 'a-price')
    price=price_class.find('span','a-offscreen')
    price=price.text
    rating=item.i.text
    review_count=item.find('span',{'class': 'a-size-base', 'dir':'auto'}).text
    result=(description,price,rating,review_count,url)
    return result
    

In [35]:
records=[]
results=soup.find_all('div',{'data-component-type': 's-search-result'})
for item in results:
    records.append(extract_record(item))

AttributeError: 'NoneType' object has no attribute 'text'

Here is eroor so we done some error handling before extrcting the data

# Eroor handling

In [36]:
def extract_record(item):
    atag=item.h2.a
    description=atag.text.strip()
    url='http://wwww.amazon.com'+atag.get('href')
    try:
        price_class=item.find('span', 'a-price')
        price=price_class.find('span','a-offscreen')
        price=price.text
    except AttributeError:
        return
    try:
        rating=item.i.text
        review_count=item.find('span',{'class': 'a-size-base', 'dir':'auto'}).text
    except AttributeError:
        rating=''
        review_count=''
    result=(description,price,rating,review_count,url)
    return result
    

In [37]:
records=[]
results=soup.find_all('div',{'data-component-type': 's-search-result'})
for item in results:
    record=extract_record(item)
    if record:
        records.append(record)

In [38]:
records[0]

('Samsung Galaxy A01 (A015M) 16GB, Dual SIM, GSM Unlocked, 5.7” Display Smartphone - International Version - Red',
 '$120.00',
 '4.4 out of 5 stars',
 '283',
 'http://wwww.amazon.com/Samsung-SM-A015M-Display-Unlocked-International/dp/B084V9FKK6/ref=sr_1_1?dchild=1&keywords=smartphone&qid=1609330837&sr=8-1')

# Getting next Page

In [53]:
def get_url(search_term):
    template='https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term=search_term.replace(' ','+')
    
    url=template.format(search_term)
    url+='&page{}'
    return url

In [54]:
url


'https:www.amazon.com'

# putting all together

In [72]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

def get_url(search_term):
    template='https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term=search_term.replace(' ','+')
    
    url=template.format(search_term)
    url+='&page{}'
    return url

def extract_record(item):
    atag=item.h2.a
    description=atag.text.strip()
    url='http://wwww.amazon.com' + atag.get('href')
    try:
        price_class=item.find('span', 'a-price')
        price=price_class.find('span','a-offscreen')
        price=price.text
    except AttributeError:
        return
    try:
        rating=item.i.text
        review_count=item.find('span',{'class': 'a-size-base', 'dir':'auto'}).text
    except AttributeError:
        rating=''
        review_count=''
    result=(description,price,rating,review_count,url)
    return result


def main(search_term):
    #startup web driver
    driver=webdriver.Chrome()
    record=[]
    url=get_url(search_term)
    
    # iterate page upto 21
    for page in range(1,21):
        driver.get(url.format(page))
        soup=BeautifulSoup(driver.page_source,'html.parser')
        results=soup.find_all('div',{'data-component-type': 's-search-result'})
        for item in results:
            record=extract_record(item)
            if record:
                records.append(record)
    driver.close()
    # save data to csv file
    with open('results.csv','w',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description','Price','Rating','Rating_count','Url'])
        writer.writerows(records)
        
    

In [73]:
main('smart phone')

In [59]:
records[2]

('BLU Advance S50 -Unlocked LTE Smartphone, 16GB+1GB RAM -Black',
 '$49.99',
 '4.0 out of 5 stars',
 '83',
 'http://wwww.amazon.com/BLU-Advance-Unlocked-Smartphone-Black/dp/B08HW1P1HP/ref=sr_1_3?dchild=1&keywords=smartphone&qid=1609330837&sr=8-3')

283
8,455
83
2,132

891
100
528
1,406
1
2,107
2,009
103
1,911
4
319
143
803
7
240
1,208
275
26
902
147
8,492
535
195
105
2,120
1,454
2,024
198
288
1,927
86
47
28
62
21
2,148
10
27

26
902
147
8,492
535
902
105
2,120
1,454
2,024
198
288
1,927
86
47
28
62
21
2,148
10
195
27
26
902
147
8,492
535
902
105
2,120
1,454
2,024
198
288
1,927
86
47
28
62
21
2,148
10
195
27
902
147
8,492
535
902
105
2,120
1,454
2,024
198
288
1,927
86
47
28
62
21
2,148
10
195
27

902
147
8,492
535
902
105
2,120
1,454
2,024
198
288
1,927
86
47
28
62
21
2,148
10
195
27

902
147
8,492
535
902
105
2,120
1,454
2,024
198
288
1,927
86
47
28
62
21
2,148
10
195
27

902
147
8,492
535
902
105
2,120
1,454
2,024
198
288
1,927
86
47
28
62
21
2,148
10
195
27

902
147
8,492
535
195
105
2,120
1,454
2,024
198
288
1,927
86
47
28
62
21
2,148
10
27

902
902
147
8,492
535
195
105
2,120
1,454
2,024
198
288
1,927
86
47
28
62
21
2,148
10
27

902
902
147
8,492
535
195
105
2,120
1,454
2,024
198
288
1,927
86
47
28
62
21
2,148
10
27

902
902
1